## 미세먼지 데이터 전처리

In [1]:
import pandas
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import pathlib
def read_csv_infolder(source_folder):
    csv_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    new_list = []
    for i,v in enumerate(csv_files):
        new_dict ={}
        if 'txt' in v:
            new_dict['sub'] = pd.read_csv(r'{}\{}'.format(source_folder,v),sep = '\t',encoding = 'utf-8')
            new_dict['title'] = v
            new_list.append(new_dict)
        elif 'csv' in v:
            new_dict['sub'] = pd.read_csv(r'{}\{}'.format(source_folder,v),sep = '|',encoding = 'utf-8')
            new_dict['title'] = v
            new_list.append(new_dict)
    new_df = pd.DataFrame(new_list)
    return new_df

### 노원구 데이터 전처리

In [2]:
### 경로를 지정
title = read_csv_infolder(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구')
### 저장된 파일이 있을시에 삭제
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_원본.csv')
if os.path.isfile(file):
    os.remove(file)
### 파일들을 하나로 합치기 
for idx in range(0, len(title.title)):
    temp = title.title[idx]
    v1 = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구\{}'.format(temp),sep=',',index_col=10)
    if file.exists() == False:
        v1.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_원본.csv', mode = 'a')
    else:
        v1.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_원본.csv', mode = 'a', header=False)

In [3]:
### 파일 불러오기
df = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_원본.csv',index_col=0)

C:\Users\dty\.conda\envs\June\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
###전처리(결측치 제거)
df= df.drop(columns = 'co2').drop(columns = 'vocs').drop(columns = 'flag')
df['pm25'] = pd.to_numeric(df.pm25)
df['pm10'] = pd.to_numeric(df.pm10)
df['tm'] = pd.to_numeric(df.tm)
df['noise'] = pd.to_numeric(df.noise)
df['humi'] = pd.to_numeric(df.humi)
df['temp'] = pd.to_numeric(df.temp)
df = df[df['noise'] != -9999]
df = df[df['temp'] != -9999.0]
df = df[df['humi'] != -9999]
df = df[df['pm25'] != -999]
df = df[df['pm10'] != -999]
df = df[df.pm25 <= 3000]  #3000 보다 큰 값들이 앞뒤의 값들과 차이가 크기 때문에 이상치로 판단
df = df.reset_index(drop=True)

### 시간별로 평균값 구하기

In [6]:
def date_h_cuter(dates):
    datelist=[]
    for idx in dates:
        date = str(idx)
        datelist.append(date[:10])
    return datelist

In [7]:
df['date_hourly'] = date_h_cuter(df.tm)
df['date_hourly'] = pd.to_datetime(df['date_hourly'], format='%Y%m%d%H', errors='ignore')
df.index = df.date_hourly
df = df.sort_index()

In [8]:
df = df.drop(columns='date_hourly')

In [10]:
###seiral 시간별 평균값 
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_pm_hourly.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in df.serial.unique():
    df_mean = df[df['serial'] == '{}'.format(idx)].reset_index().groupby('date_hourly').mean().reset_index().copy().drop(columns = 'tm')
    df_mean['serial'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_pm_hourly.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_pm_hourly.csv', mode = 'a', header=False)

In [11]:
data_hourly = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_pm_hourly.csv', index_col=0).reset_index(drop=True)

### 센서를 동별로 분류

In [19]:
station = """V10O1610610
V10O1610376
V10O1610293
V10O1610356
V10O1610616
V10O1610200
V10O1610643
V10O1610642
V10O1610297
V10O1610312
V10O1610102
V10O1610351
V10O1610629
V10O1610630
V10O1611104
V10O1611097
V10O1611100
V10O1612126
V10O1611102
V10O1611652
V10O1611150
V10O1611229"""
station= station.split('\n')

In [20]:
station_kor ="""하계1동
상계6,7동
상계10동
상계1동
상계2동
상계2동
월계3동
공릉2동
상계5동
상계2동
하계1동
상계6,7동
공릉1동
공릉1동
상계6,7동
중계2,3동
상계2동
중계본동
상계5동
공릉2동
상계3,4동
월계1동"""
station_kor= station_kor.split('\n')

In [21]:
station_with_names = []
for idx in range(0, len(station)):
    station_with_names.append([station[idx],station_kor[idx]])

In [22]:
data_hourly['동'] = ''

In [23]:
###serial기반 동 컬럼 추가
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v1.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in data_hourly.serial.unique():
    for idx2 in station_with_names:
        if idx in idx2:
            temp = data_hourly[data_hourly['serial']==str(idx)]
            temp['동'] = idx2[1]
            if file.exists() == False:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v1.csv', mode = 'a')
            else:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v1.csv', mode = 'a', header=False)

C:\Users\dty\.conda\envs\June\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
data_hourly = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v1.csv', index_col = 0)

In [29]:
###동별 시간별 평균값
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v2.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in data_hourly.동.unique():
    df_mean = data_hourly[data_hourly['동'] == '{}'.format(idx)].groupby('date_hourly').mean().reset_index().copy()
    df_mean['동'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v2.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v2.csv', mode = 'a', header=False)

In [34]:
data_hourly = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_hourly_동_v2.csv', index_col = 0).reset_index(drop=True)

### 일별로 평균값 구하기

In [37]:
def date_cuter(dates):
    datelist=[]
    for idx in dates:
        date = str(idx)
        datelist.append(date[:8])
    return datelist

In [38]:
df['date_daily'] = date_cuter(df.tm)
df.index = df.date_daily
df = df.sort_index()
df = df.drop(columns = 'date_daily')

In [39]:
### serial 일별 평균값 
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_daily_v1_08_22.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in df.serial.unique():
    df_mean = df[df['serial'] == '{}'.format(idx)].reset_index().groupby('date_daily').mean().reset_index().copy().drop(columns = 'tm')
    df_mean['serial'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_daily_v1_08_22.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_daily_v1_08_22.csv', mode = 'a', header=False)

In [40]:
date_daily = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\노원구_daily_v1_08_22.csv', index_col=0).reset_index(drop=True)

In [41]:
date_daily['동'] = ''

In [44]:
###serial기반 동 컬럼 추가
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v1.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in date_daily.serial.unique():
    for idx2 in station_with_names:
        if idx in idx2:
            temp = date_daily[date_daily['serial']==str(idx)]
            temp['동'] = idx2[1]
            if file.exists() == False:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v1.csv', mode = 'a')
            else:
                temp.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v1.csv', mode = 'a', header=False)

C:\Users\dty\.conda\envs\June\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [45]:
date_daily = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v1.csv', index_col = 0)

In [48]:
###동별 일별 평균값
file = pathlib.Path(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v2.csv')
if os.path.isfile(file):
    os.remove(file)
for idx in date_daily.동.unique():
    df_mean = date_daily[date_daily['동'] == '{}'.format(idx)].groupby('date_daily').mean().reset_index().copy()
    df_mean['동'] = idx
    if file.exists() == False:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v2.csv', mode = 'a')
    else:
        df_mean.to_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v2.csv', mode = 'a', header=False)

In [51]:
date_daily = pd.read_csv(r'C:\Users\dty\Desktop\미세먼지_공모전용\미세먼지_daily_동_v2.csv', index_col = 0).reset_index(drop=True)

In [52]:
date_daily

,date_daily,pm10,noise,temp,humi,pm25,동
0,20180401,66.173190,43.212724,16.750779,57.037444,34.259229,상계5동
1,20180402,67.927270,43.475027,19.843902,58.208500,28.146785,상계5동
2,20180403,69.850694,43.329861,19.471146,61.844792,23.350347,상계5동
3,20180404,18.310493,44.169948,13.908589,61.093240,11.546391,상계5동
4,20180405,12.328449,44.779427,8.660138,76.031968,4.514871,상계5동
5,20180406,85.023958,44.463889,7.788090,62.387500,31.271875,상계5동
6,20180407,42.200347,43.912500,6.107222,35.537153,20.613889,상계5동
7,20180408,29.734206,43.689257,4.897619,52.013606,14.741279,상계5동
8,20180409,42.640625,43.969444,8.146632,54.778472,29.408681,상계5동
9,20180410,49.465234,44.017059,15.756773,46.305708,20.978574,상계5동
